In [39]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

In [2]:
dataset = np.loadtxt('C:\\Users\\aarus\\Downloads\\archive\\A_Z Handwritten Data.csv', delimiter=',')

In [6]:
X = dataset[:, 0:784]  # Extract columns 1 to 784 as input features
Y = dataset[:, 0]     # Extract the first column as the target variable

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X ,Y, test_size=0.3,random_state=42)

# Resizing and preprocessing

In [8]:
X_train=X_train.reshape(X_train.shape[0],28,28,1).astype('float32')

In [9]:
X_test=X_test.reshape(X_test.shape[0],28,28,1).astype('float32')

# Normalisation

In [10]:
X_train = X_train / 255
X_test = X_test / 255

# Splitting into categories

In [11]:
from sklearn.preprocessing import LabelBinarizer

# Instantiate the LabelBinarizer
label_binarizer = LabelBinarizer()

# Fit and transform the training labels
Y_train_onehot = label_binarizer.fit_transform(Y_train)

# Transform the test labels (no need to fit again)
Y_test_onehot = label_binarizer.transform(Y_test)

In [12]:
num_classes = Y_train_onehot.shape[1]

In [13]:
X_train

array([[[[0.05490196],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        ...,

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]]],


       [[[0.07058824],
         [0.        ],
         [0.  

# Model Making 

In [41]:
import tensorflow as tf
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D


# CNN architecture

In [46]:
from keras import backend as K

In [47]:
model=Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 24, 24, 32)        832       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [48]:
Train1=model.fit(X_train,Y_train_onehot,batch_size=128, epochs=5)

Epoch 1/5
2037/2037 [==============================] - 141s 68ms/step - loss: 0.1855 - accuracy: 0.9470
Epoch 2/5
2037/2037 [==============================] - 137s 67ms/step - loss: 0.0941 - accuracy: 0.9736
Epoch 3/5
2037/2037 [==============================] - 142s 70ms/step - loss: 0.0864 - accuracy: 0.9756
Epoch 4/5
2037/2037 [==============================] - 142s 70ms/step - loss: 0.0825 - accuracy: 0.9768
Epoch 5/5
2037/2037 [==============================] - 142s 69ms/step - loss: 0.0814 - accuracy: 0.9772


In [51]:
pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.Collecting keras-tuner
                                              0.0/127.5 kB ? eta -:--:--
     ---------                               30.7/127.5 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 127.5/127.5 kB 1.5 MB/s eta 0:00:00
                                              0.0/950.8 kB ? eta -:--:--
     ------                                 153.6/950.8 kB 4.6 MB/s eta 0:00:01
     ------------                           307.2/950.8 kB 3.8 MB/s eta 0:00:01
     --------------------                   522.2/950.8 kB 4.1 MB/s eta 0:00:01
     ------------------------------         768.0/950.8 kB 4.4 MB/s eta 0:00:01
     -------------------------------------  942.1/950.8 kB 4.3 MB/s eta 0:00:01
     -------------------------------------- 950.8/950.8 kB 4.0 MB/s eta 0:00:00
                                              0.0/101.3 kB ? eta -:--:--
     -------------------------------------- 101.3/101


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Tuning Hyperparameters

In [14]:
import keras
from keras_tuner import HyperParameters

Using TensorFlow backend


In [15]:
def build_model(hp):
    model = keras.Sequential()

    # Add the first convolutional layer
    model.add(keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
        activation='relu',
        input_shape=(28, 28, 1)
    ))

    # Add a pooling layer after the first convolutional layer
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    
    
    model.add(keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=64, max_value=128, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
        activation='relu'
    ))

    # Add a pooling layer after the second convolutional layer
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Flatten())
 
    # Add the first dense layer
    model.add(keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ))
    model.add(keras.layers.Dense(26, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [16]:
from kerastuner import RandomSearch
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=3,directory='output',project_name="alphabet")

C:\Users\aarus\AppData\Local\Temp\ipykernel_312\536598455.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


Reloading Tuner from output\alphabet\tuner0.json


In [17]:
tuner_search.search(X_train,Y_train,epochs=3,validation_split=0.1)

Trial 4 Complete [00h 25m 09s]
val_accuracy: 0.9845427870750427

Best val_accuracy So Far: 0.9856551289558411
Total elapsed time: 00h 25m 09s


In [23]:
model=tuner_search.get_best_models(num_models=1)[0]

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 80)        23120     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 80)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 2000)              0         
                                                                 
 dense (Dense)               (None, 48)                9

In [28]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Tune hyperparameters
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=0.1, sampling='log')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [34]:
from kerastuner.tuners import RandomSearch

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',  # You can choose the metric to optimize
    max_trials=2,  # Number of different hyperparameter combinations to try
    directory='my_tuning_directory',  # Directory to store results
    project_name='my_model_tuning'  # Name for this tuning project
)


Reloading Tuner from my_tuning_directory\my_model_tuning\tuner0.json


In [35]:
tuner.search(X_train, Y_train_onehot, epochs=2, validation_data=(X_test, Y_test_onehot))

Trial 2 Complete [00h 11m 18s]
val_accuracy: 0.9871840476989746

Best val_accuracy So Far: 0.9892603754997253
Total elapsed time: 00h 11m 18s


In [36]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = build_model(best_hyperparameters)

In [48]:
best_model.fit(X_train, Y_train_onehot, epochs=3, batch_size=128)

Epoch 1/5
 683/2037 [=========>....................] - ETA: 42s - loss: 0.0417 - accuracy: 0.9879

KeyboardInterrupt: 

In [41]:
best_model.save(r'model_i_made.h5')

In [46]:
test_loss, test_accuracy = best_model.evaluate(X_test, Y_test_onehot)
print("Test Accuracy:", test_accuracy)

3492/3492 [==============================] - 39s 11ms/step - loss: 0.0547 - accuracy: 0.9851
Test Accuracy: 0.985116720199585


In [47]:
test_loss

0.05472322925925255